In [ ]:
VER = 11

W_VER = 22

In [ ]:
from collections import defaultdict
import pandas as pd
from tqdm.notebook import tqdm
import glob
import numpy as np
import multiprocessing
import os
import pickle

import glob
from collections import Counter

ITEM_CT = 50

## Test set inference

In [ ]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}
def load_test():    
    dfs = []
    for e, chunk_file in enumerate(tqdm(glob.glob('/raid/Kaggle/otto/parquet/test_parquet/*'))):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)

    return pd.concat(dfs).reset_index(drop=True)#.astype({"ts": "datetime64[ms]"})

In [ ]:
#test_df = load_test()

In [ ]:
# COMPUTED FROM TEST DATA
top_orders = [ 986164, 1460571,  329725, 1043508,  332654,  688602,   29735,
       1495817,  579690, 1022566, 1006198,  471073,  832192,  544144,
       1825743,  836852,  147526, 1236775,  166037, 1030009, 1609228,
        508883,  923948, 1462420,  892871,  554660, 1457846,  258353,
       1734475,  480314,  145332,  108125, 1286213, 1336175, 1359971,
        137514,  714524,  558573,  172856,  585186,  352192, 1176975,
       1146575,  954951, 1496287,  823143, 1699089,   25964, 1257293,
        399315, 1441266, 1196256, 1294924, 1603001, 1274545,  414968,
       1581568,  247240, 1116095,  383437,  530377,  272744, 1445562,
        269257,  791627, 1140985, 1708326,  631899,  670066,  122983,
        223273,  165160,  881286, 1768724,  868327, 1604220,  406358,
       1722991, 1568011, 1025795, 1647563,  835431, 1531805,  714968,
        500609, 1217083, 1668343, 1159757, 1610239, 1647157, 1264313,
       1798916,  423558,  752652,  184976, 1255910, 1413049,  801774,
        615566, 1034578]
top_carts = [ 485256,   33343, 1460571,  986164,  554660,  660655, 1116095,
        152547, 1022566,  544144,  832192,  579690,  329725, 1043508,
       1006198,  558573,  471073,  332654,  688602,   29735,  508883,
        258353, 1736857, 1462420,  166037, 1609228, 1778843,  108125,
       1495817, 1604220, 1825743, 1562705,  147526,  836852, 1286213,
         25964, 1236775,  923948, 1281615, 1257293,  917587,  835431,
       1439409,  892871,  125957,  122983, 1097061, 1449873, 1568011,
       1030727, 1146575, 1731920,  326904, 1196256,  714524, 1768724,
        480314, 1800674, 1662401, 1359971,  455191,  496180,  145332,
        616283, 1708326, 1294924, 1270528,  944778, 1223508,  881286,
        165160,  272744,  670066,  868327, 1734475,  137514,  172856,
       1122221,  442293, 1685214,  823143, 1413049, 1722991, 1647157,
        406358, 1733943,  700995, 1025795,  754412,  530377,  102416,
        184976, 1445562, 1565495, 1019736, 1274545, 1083665,  667563,
       1264313,  563117]
top_clicks = [1460571,  485256,  108125,  986164, 1551213,  754412,  554660,
        832192,  579690,   33343, 1006198,  688602,   29735,  329725,
        184976, 1019736,  496180,  861401,  944778,  659399, 1043508,
       1022566,  811371, 1604220,  836852,  471073,  819288, 1264313,
        508883, 1751274,  620545,  959208,  717965,  332654, 1731920,
        544144,  147526, 1116095, 1294924,  102345, 1645990, 1497089,
        558573,   95488, 1196256,  199409, 1110150, 1146575, 1236775,
        137514, 1030009,  435253, 1800674,  881286, 1609228, 1286213,
        337471,  670066,  831165, 1685214, 1673641,  909449, 1260564,
       1099100,  995962,  612920, 1647563, 1462420, 1741695, 1281615,
       1603001, 1722991,  442293,  206735, 1219503,  166037,  799923,
       1469891,  557072, 1156699,  111891, 1624436, 1782099, 1639229,
        530377, 1197632, 1140985,  152547,  247240, 1449873, 1825743,
        901817, 1420240, 1733943,  542343,  680375,  406358,  147278,
       1627951,  836707]

In [ ]:
%%time

top_20 = pickle.load(open('../../data/covisit_matrices/top_40_aids_v104.pkl', 'rb')) #23
for k,v in top_20.items():
    top_20[k] = [np.int32(x) for x in v[:23]]

top_20b = pickle.load(open('../../data/covisit_matrices/top_40_aids_v23.pkl', 'rb')) #23
for k,v in top_20b.items():
    top_20b[k] = [np.int32(x) for x in v[:23]]
top_20c = pickle.load(open('../../data/covisit_matrices/top_80_aids_v24.pkl', 'rb')) #55
for k,v in top_20c.items():
    top_20c[k] = [np.int32(x) for x in v[:55]]
top_20d = pickle.load(open('../../data/covisit_matrices/top_30_aids_v28.pkl', 'rb')) #20
for k,v in top_20d.items():
    top_20d[k] = [np.int32(x) for x in v[:20]]
    
top_20e = pickle.load(open('../../data/covisit_matrices/top_80_aids_v130.pkl', 'rb')) #55 #UPGRADED
for k,v in top_20e.items():
    top_20e[k] = [np.int32(x) for x in v[:55]]
    
top_20f = pickle.load(open('../../data/covisit_matrices/top_80_aids_v132.pkl', 'rb')) #10 #NEW
for k,v in top_20f.items():
    top_20f[k] = [np.int32(x) for x in v[:10]]
    
top_20_test2 = pickle.load(open('../../data/covisit_matrices/top_40_aids_v34.pkl', 'rb')) #23
for k,v in top_20_test2.items():
    top_20_test2[k] = [np.int32(x) for x in v[:23]]

In [ ]:
import gc
gc.collect()

In [ ]:
INCLUDE = ['top20','top20b','top20c','top20d','top20e','top20f','top20orders','top20buy2buy','top20buy2buy2',
          'top20test','top20test2','top20buy']
INFER = ['top20buy2buy','top20buy2buy2']

In [ ]:
import itertools

type_weight_multipliers = {0: 1, 1: 6, 2: 3}

def suggest_aids(df):
    aids=df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    
    df2 = df.sort_values('ts',ascending=False).drop_duplicates('d')
    aids2 = df2.aid.tolist()
    unique_aids3 = list(dict.fromkeys(aids2[::-1] )) #last of each session
    
    mx = df.d.max()
    aids2 = df.loc[df.d==mx].aid.tolist()
    unique_aids4 = list(dict.fromkeys(aids2[::-1] ))
    
    df = df.loc[ df['type'].isin([1,2]) ]
    unique_buys = list(dict.fromkeys( df.aid.tolist()[::-1] ))
    
    ln = len(unique_aids)
 
    if 1: #len(unique_aids)>=20:
        weights=np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        if 'top20c' in INFER:
            aids3 = list(itertools.chain(*[top_20c[aid][:20] for aid in unique_aids[:2] if aid in top_20c]))
            for i,aid in enumerate(aids3):
                aids_temp[aid] += 0.6
        if 'top20b' in INFER:
            aids3 = list(itertools.chain(*[top_20b[aid][:15] for aid in unique_aids3 if aid in top_20b]))
            for i,aid in enumerate(aids3):
                aids_temp[aid] += 0.3
        if 'top20test2' in INFER:
            aids3 = list(itertools.chain(*[top_20_test2[aid][:20] for aid in unique_aids[:2] if aid in top_20_test2]))
            for i,aid in enumerate(aids3):
                aids_temp[aid] += 0.6
                
        sorted_aids = [k for k,v in aids_temp.most_common(ITEM_CT)]
        return (sorted_aids + top_clicks[:ITEM_CT-len(sorted_aids)])[:ITEM_CT]
        #return sorted_aids 
    
    aids_temp = Counter() 
    if 'top20' in INFER:
        aids2 = list(itertools.chain(*[top_20[aid][:20] for aid in unique_aids if aid in top_20]))
        for i,aid in enumerate(aids2):
            m = 0.1 + 0.9*(ln-(i//20))/ln
            aids_temp[aid] += m
            if i%20==0: aids_temp[aid] += m
    if 'top20b' in INFER:
        aids3 = list(itertools.chain(*[top_20b[aid][:20] for aid in unique_aids[:2] if aid in top_20b]))
        for i,aid in enumerate(aids3):
            aids_temp[aid] += 1
            if i%20==0: aids_temp[aid] += 1
    if 'top20test2' in INFER:            
        aids3 = list(itertools.chain(*[top_20_test2[aid][:20] for aid in unique_aids[:2] if aid in top_20_test2]))
        for i,aid in enumerate(aids3):
            aids_temp[aid] += 1
            if i%20==0: aids_temp[aid] += 1
    if 'top20f' in INFER:            
        aids4 = list(itertools.chain(*[top_20f[aid][:10] for aid in unique_aids4 if aid in top_20f]))
        for i,aid in enumerate(aids4):
            w = i//10
            aids_temp[aid] += 1 -w*0.1
            if i%10==0: aids_temp[aid] += 1 -w*0.1
    if 'top20e' in INFER:             
        aids5 = list(itertools.chain(*[top_20e[aid][:20] for aid in unique_aids3 if aid in top_20e]))
        for i,aid in enumerate(aids5):
            aids_temp[aid] += 1
            if i%20==0: aids_temp[aid] += 1
        top_aids2 = [k for k,v in aids_temp.most_common(1) if k not in unique_aids]
    if 'top20c' in INFER:    
        aids3 = list(itertools.chain(*[top_20c[aid][:10] for aid in top_aids2 if aid in top_20c]))
        for i,aid in enumerate(aids3):
            aids_temp[aid] += 1
            if i%10==0: aids_temp[aid] += 1
        top_aids2 = [k for k,v in aids_temp.most_common(ITEM_CT) if k not in unique_aids]
    
    result = unique_aids + top_aids2[:ITEM_CT - len(unique_aids)]
    return (result + top_clicks[:ITEM_CT-len(result)])[:ITEM_CT]

def suggest_orders(df):
    aids = df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    
    mx = df.d.max()
    aids2 = df.loc[df.d==mx].aid.tolist()
    unique_aids4 = list(dict.fromkeys(aids2[::-1] ))
    
    mx = df.ts.max()
    aids2 = df.loc[df.ts>=mx - 60*60/2].aid.tolist()
    unique_aids5 = list(dict.fromkeys(aids2[::-1] )) #recent 1/2 hour   
    
    df2 = df.drop_duplicates('d')
    aids2 = df2.aid.tolist()
    unique_aids2 = list(dict.fromkeys(aids2[::-1] )) #first of each session
    
    df2 = df.sort_values('ts',ascending=False).drop_duplicates('d')
    aids2 = df2.aid.tolist()
    unique_aids3 = list(dict.fromkeys(aids2 )) #last of each session
    
    df = df.loc[ df['type'].isin([1,2]) ]
    unique_buys = list(dict.fromkeys( df.aid.tolist()[::-1] ))
    
    if 1: #len(unique_aids)>=20:

        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        for aid in unique_aids2: 
            aids_temp[aid] += 0.5
        for aid in unique_aids3: 
            aids_temp[aid] += 0.5
        if 'top20buy2buy' in INFER:
            aids3 = list(itertools.chain(*[top_20_buy2buy[aid][:40] for aid in unique_buys if aid in top_20_buy2buy]))
            for i,aid in enumerate(aids3):
                aids_temp[aid] += 0.05
                if i%40==0: aids_temp[aid] += 0.05
        if 'top20buy2buy2' in INFER:
            aids3 = list(itertools.chain(*[top_20_buy2buy2[aid][:40] for aid in unique_buys if aid in top_20_buy2buy2]))
            for i,aid in enumerate(aids3):
                aids_temp[aid] += 0.1
                if i%40==0: aids_temp[aid] += 0.1
        if 'top20test' in INFER:             
            aids4 = list(itertools.chain(*[top_20_test[aid][:40] for aid in unique_aids if aid in top_20_test]))
            for i,aid in enumerate(aids4):
                aids_temp[aid] += 0.05
                if i%40==0: aids_temp[aid] += 0.05
        if 'top20c' in INFER:
            aids5 = list(itertools.chain(*[top_20c[aid][:20] for aid in unique_aids[:1] if aid in top_20c]))
            for i,aid in enumerate(aids5):
                aids_temp[aid] += 0.05
                if i%20==0: aids_temp[aid] += 0.05
        if 'top20d' in INFER:
            aids6 = list(itertools.chain(*[top_20d[aid][:20] for aid in unique_buys[:1] if aid in top_20d]))
            for i,aid in enumerate(aids6):
                aids_temp[aid] += 0.05
                if i%20==0: aids_temp[aid] += 0.05
        if 'top20b' in INFER:  
            aids7 = list(itertools.chain(*[top_20b[aid][:5] for aid in unique_aids3 if aid in top_20b]))
            for i,aid in enumerate(aids7):
                aids_temp[aid] += 0.25
                if i%5==0: aids_temp[aid] += 0.25
            aids7 = list(itertools.chain(*[top_20b[aid][:5] for aid in unique_aids2 if aid in top_20b]))
            for i,aid in enumerate(aids7):
                aids_temp[aid] += 0.125
                if i%5==0: aids_temp[aid] += 0.125
            
        sorted_aids = [k for k,v in aids_temp.most_common(ITEM_CT)]
        return (sorted_aids + top_orders[:ITEM_CT-len(sorted_aids)])[:ITEM_CT]
        #return sorted_aids 
    
    weights = [2,2] + [1]*8 #+ [0]*30
    weights2 = [2,2] + [1]*53 #+ [0]*25
    weights3 = [2,2] + [1]*18 #+ [0]*70
    weights4 = [2,2] + [1]*38 #+ [0]*70
    
    ln = len(unique_aids)
    
    aids_temp = Counter() 
    if 'top20orders' in INFER:
        aids2 = list(itertools.chain(*[top_20_orders[aid][:10] for aid in unique_aids if aid in top_20_orders]))
        w2 = weights* int(len(aids2)//10)
        for i,(aid,w) in enumerate(zip(aids2,w2)):
            m = 0.25 + 0.75*(ln-(i//10))/ln
            aids_temp[aid] += w*m
    if 'top20buy2buy' in INFER:
        aids3 = list(itertools.chain(*[top_20_buy2buy[aid][:10] for aid in unique_buys if aid in top_20_buy2buy]))
        w3 = weights* int(len(aids3)//10)
        for i,(aid,w) in enumerate(zip(aids3,w3)):
            aids_temp[aid] += w/2
    if 'top20test' in INFER:
        aids4 = list(itertools.chain(*[top_20_test[aid][:10] for aid in unique_aids if aid in top_20_test]))
        w4 = weights* int(len(aids4)//10)
        for i,(aid,w) in enumerate(zip(aids4,w4)):
            m = 0.25 + 0.75*(ln-(i//10))/ln
            aids_temp[aid] += w*m
    if 'top20buy2buy2' in INFER:
        aids5 = list(itertools.chain(*[top_20_buy2buy2[aid][:10] for aid in unique_buys if aid in top_20_buy2buy2]))
        w5 = weights* int(len(aids5)//10)
        for i,(aid,w) in enumerate(zip(aids5,w5)):
            aids_temp[aid] += w/2
    if 'top20c' in INFER:        
        aids5 = list(itertools.chain(*[top_20c[aid][:55] for aid in unique_aids[:1] if aid in top_20c]))
        w5 = weights2* int(len(aids5)//55)
        for aid,w in zip(aids5,w5):
            aids_temp[aid] += w
    if 'top20d' in INFER:
        aids5 = list(itertools.chain(*[top_20d[aid][:20] for aid in unique_buys[:1] if aid in top_20d]))
        w5 = weights3* int(len(aids5)//20)
        for aid,w in zip(aids5,w5):
            aids_temp[aid] += w
        
    # NEW
    ln2 = len(unique_aids5)
    if 'top20buy' in INFER:
        aids5 = list(itertools.chain(*[top_20_buy[aid][:20] for aid in unique_aids5 if aid in top_20_buy]))
        w5 = weights3* int(len(aids5)//20)
        for aid,w in zip(aids5,w5):
            aids_temp[aid] += 2*w/ln2
    if 'top20f' in INFER:         
        aids4 = list(itertools.chain(*[top_20f[aid][:5] for aid in unique_aids4 if aid in top_20f]))
        for i,aid in enumerate(aids4):
            w = i//5
            aids_temp[aid] += 1/2 -w*0.05
            if i%5==0: aids_temp[aid] += 1/2 -w*0.05
    if 'top20e' in INFER:
        aids5 = list(itertools.chain(*[top_20e[aid][:55] for aid in unique_aids3 if aid in top_20e]))
        w5 = weights2* int(len(aids5)//55)
        for i,(aid,w) in enumerate(zip(aids5,w5)):
            w2 = i//55
            aids_temp[aid] += w -w2*0.1
    if 'top20e' in INFER:
        aids5 = list(itertools.chain(*[top_20e[aid][:10] for aid in unique_aids2 if aid in top_20e]))
        w5 = weights* int(len(aids5)//10)
        for i,(aid,w) in enumerate(zip(aids5,w5)):
            w2 = i//10
            aids_temp[aid] += w/2. -w2*0.05
                    
    sorted_aids = [k for k,v in aids_temp.most_common(ITEM_CT) if k not in unique_aids]
    
    result = unique_aids + sorted_aids[:ITEM_CT - len(unique_aids)]
    return (result + top_orders[:ITEM_CT-len(result)])[:ITEM_CT]

In [ ]:
%%time
if 0:
    test_df = test_df.sort_values(["session", "ts"])
    test_df['d'] = test_df.groupby('session').ts.diff()
    test_df.d = (test_df.d > 60*60*2).astype('int16').fillna(0)
    test_df.d = test_df.groupby('session').d.cumsum()

    test_df.to_parquet('test_LB.pqt',index=False)
else:
    test_df = pd.read_parquet('test_LB.pqt')

In [ ]:
#from pandarallel import pandarallel
#pandarallel.initialize(progress_bar=True, nb_workers=8, use_memory_fs=True)

In [ ]:
#%%time
#pred_df = test_df.groupby(["session"]).parallel_apply(
#    lambda x: suggest_aids(x)
#)
del top_20
gc.collect()

In [ ]:
%%time
top_20_orders = pickle.load(open('../../data/covisit_matrices/top_40_orders_carts_v118.pkl', 'rb')) #13 #UPGRADED
for k,v in top_20_orders.items():
    top_20_orders[k] = [np.int32(x) for x in v[:13]]
top_20_carts = top_20_orders 

top_20_buy2buy = pickle.load(open('../../data/covisit_matrices/top_40_buy2buy_v16.pkl', 'rb'))
for k,v in top_20_buy2buy.items():
    top_20_buy2buy[k] = [np.int32(x) for x in v]
    
top_20_buy2buy2 = pickle.load(open('../../data/covisit_matrices/top_40_buy2buy_v119.pkl', 'rb')) #NEW
for k,v in top_20_buy2buy2.items():
    top_20_buy2buy2[k] = [np.int32(x) for x in v]
    
top_20_test = pickle.load(open('../../data/covisit_matrices/top_40_aids_v35.pkl', 'rb'))
for k,v in top_20_test.items():
    top_20_test[k] = [np.int32(x) for x in v]

top_20_buy = pickle.load(open('../../data/covisit_matrices/top_20_aids_v37.pkl', 'rb'))
for k,v in top_20_buy.items():
    top_20_buy[k] = [np.int32(x) for x in v]
    
gc.collect()

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
%%time
pred_df_orders = test_df.groupby(["session"]).progress_apply(
    lambda x: suggest_orders(x)
)
#pred_df_carts = pred_df_orders

In [ ]:
del top_20_orders, top_20_carts, top_20_buy2buy, top_20_test, top_20b, top_20c, top_20d, top_20e
gc.collect()

In [ ]:
#clicks_pred_df = pd.DataFrame(pred_df.add_suffix("_clicks"), columns=["labels"]).reset_index()
pred_df = pd.DataFrame(pred_df_orders.add_suffix("_orders"), columns=["labels"]).reset_index()
#carts_pred_df = pd.DataFrame(pred_df_carts.add_suffix("_carts"), columns=["labels"]).reset_index()

In [ ]:
#pred_df

In [ ]:
#pred_df = pd.concat(
#    [clicks_pred_df, orders_pred_df, carts_pred_df]
#)
pred_df.columns = ["session_type", "labels"]
pred_df["labels"] = pred_df.labels.apply(lambda x: " ".join(map(str,x)))
pred_df.to_csv(f"../../data/candidate_scores/submission_v{VER}_v{W_VER}.csv", index=False)